In [ ]:
%env kubectl=/Users/gsjurseth/GoogleShtuff/google-cloud-sdk/bin/kubectl
%env istio=/Users/gsjurseth/istio-1.0.2
%env istioctl=/Users/gsjurseth/istio-1.0.2/bin/istioctl
%env ingress=35.225.166.160
%env demo=/Users/gsjurseth/dev/fazio/microservices-demo
%env apigee-istio=/Users/gsjurseth/istio/apigee-istio
%env mypass=`dapass`

---
## Expose the tracking api

In [ ]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  annotations:
  name: frontend-ingress
  namespace: default
spec:
  gateways:
  - frontend-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        exact: /
    - uri:
        prefix: /cart
    - uri:
        prefix: /product
    - uri:
        exact: /setCurrency
    - uri:
        prefix: /static
    route:
    - destination:
        host: frontend
        port:
          number: 80
  - match:
    - uri:
        prefix: /tracking
    route:
    - destination:
        host: shipping
        port:
          number: 8080
    corsPolicy:
      allowOrigin:
      - "*"
      allowMethods:
      - GET
      - POST
      allowHeaders:
      - content-type
      - x-api-key
      allowCredentials: true

EOF

---
## Access the API

In [ ]:
!curl $ingress/tracking/1 | jq .

---
## Wire up Apigee

The following configures how Istio passes data to the Apigee adapter.

In [148]:
%%bash
cat <<EOF | $kubectl apply -f -

# instance configuration for template 'apigee.analytics'
apiVersion: config.istio.io/v1alpha2
kind: analytics
metadata:
  name: apigee
  namespace: istio-system
spec:
  api_key: request.api_key | request.headers["x-api-key"] | ""
  api_proxy: api.service | destination.service.host | ""
  response_status_code: response.code | 0
  client_ip: source.ip | ip("0.0.0.0")
  request_verb: request.method | ""
  request_uri: request.path | ""
  useragent: request.useragent | ""
  client_received_start_timestamp: request.time
  client_received_end_timestamp: request.time
  target_sent_start_timestamp: request.time
  target_sent_end_timestamp: request.time
  target_received_start_timestamp: response.time
  target_received_end_timestamp: response.time
  client_sent_start_timestamp: response.time
  client_sent_end_timestamp: response.time
  api_claims: # from jwt
    json_claims: request.auth.raw_claims | ""
---

# instance configuration for template 'apigee.authorization'
apiVersion: config.istio.io/v1alpha2
kind: authorization
metadata:
  name: apigee
  namespace: istio-system
spec:
  subject:
    user: ""
    groups: ""
    properties:
      api_key: request.api_key | request.headers["x-api-key"] | ""
      json_claims: request.auth.raw_claims | ""
  action:
    namespace: destination.namespace | "default"
    service: api.service | destination.service.host | ""
    path: api.operation | request.path | ""
    method: request.method | ""

EOF

analytics.config.istio.io "apigee" configured
authorization.config.istio.io "apigee" configured


---
## Analytics Rule

This lets Istio know that we want to send all Istio telemetry to Apigee via the adapter.

In [149]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-analytics
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
    - apigee.analytics

EOF

rule.config.istio.io "apigee-analytics" configured


## Analytics Reporting

Now we have analytics coming from our Istio services.

https://apigee.com/platform/emea-poc15/devices

---
## Apigee API Product

An API Product is how I will expose the API to developers.

https://apigee.com/platform/emea-poc15/products/Tracking%20Details

---
## Protect the API with an API Key

In [150]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-authorization
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound" && destination.service.name == "shipping"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
    - apigee.analytics
    - apigee.authorization

EOF

rule.config.istio.io "apigee-authorization" configured


---
## Create an App to access the API Product

The Developer Portal lets our developers create their own Apps to access protected API Product.

https://emea-poc15-fazioapiportal.apigee.io/

--- 
## Now we have an API Key .. Let's try hitting the api one more time with the key

---
### Check the result

In [154]:
!curl http://$ingress/tracking/123 -H "x-api-key: TiZKngjgrJxHqCRblIGvQgaKSvZihfGR" | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   359  100   359    0     0    341      0  0:00:01  0:00:01 --:--:--   341
{
  "id": "123",
  "tracking_code": "fbdf02b0-ce1d-11e8-bb8a-dd6df18ddb56",
  "status": 39,
  "created": "Fri, 12 Oct 2018 12:54:40 GMT",
  "updated": "Fri, 12 Oct 2018 12:54:40 GMT",
  "signed": "John Doe",
  "weight": "21.61",
  "estiamted_delivery_date": "Fri, 12 Oct 2018 12:54:40 GMT",
  "carrier": "USPS",
  "trackingLocation": {
    "city": "Los Angeles",
    "state": "CA",
    "country": "US",
    "zip": "90001"
  }
}


---
## Rollback APIKey protection

In [ ]:
%%bash
cat <<EOF | $kubectl apply -f -

apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-authorization
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound" && destination.service.name == "shipping"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
        - apigee.analytics


EOF

In [151]:
!curl -i http://$ingress/tracking/123

HTTP/1.1 403 Forbidden
content-length: 75
content-type: text/plain
date: Fri, 12 Oct 2018 12:53:58 GMT
server: envoy
x-envoy-upstream-service-time: 5

PERMISSION_DENIED:apigee-handler.apigee.istio-system:missing authentication